# **Installing Dependencies**

In [ ]:
! pip install transformers peft trl datasets pandas accelerate torch

In [2]:
from huggingface_hub import login
login(token="") 

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\Siddharth\.cache\huggingface\token
Login successful


# **Data preprocessing**

In [8]:
from datasets import load_dataset , Dataset
dataset = load_dataset('parthrautV/gemma_agri_dataset')

In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 21335
    })
})

In [10]:
import pandas as pd
df = pd.DataFrame(dataset["train"])
df

,text
0,<start_of_turn>What is the disease called when...
1,<start_of_turn>How many types of sigatoka are ...
2,<start_of_turn>Which species of sigatoka is mo...
3,<start_of_turn>Where is the leaf spot of banan...
4,<start_of_turn>Which variety of banana is high...
...,...
21330,<start_of_turn>What is the purpose of seed tre...
21331,<start_of_turn>Is TV harmful to plants or anim...
21332,<start_of_turn>What is the process of seed tre...
21333,<start_of_turn>How is seedling treatment done?...


In [15]:
df.head(1)

,text
0,<start_of_turn>What is the disease called when...


In [11]:
df.to_csv("first_data.csv",index=False)

In [5]:
from datasets import load_dataset
dataset2 = load_dataset("KisanVaani/agriculture-qa-english-only")
dataset2

DatasetDict({
    train: Dataset({
        features: ['question', 'answers'],
        num_rows: 22615
    })
})

In [6]:
df2 = pd.DataFrame(dataset2["train"])
df2

,question,answers
0,why is crop rotation important in farming?,This helps to prevent soil erosion and depleti...
1,What farming practice helps prevent soil erosion?,Crop Rotation
2,what is crop rotation,Crop rotation is the practice of growing a ser...
3,what are the different methods of irrigation?,"surface irrigation, drip irrigation, and sprin..."
4,why is soil health vital?,Soil health is critical to crop growth and pro...
...,...,...
22610,Does cassava mosaic affect the cassava stems?,"Yes, it makes them small and not good for plan..."
22611,How does the pathogen of Phoma Rot spread?,The pathogen of Phoma Rot is seed-borne.
22612,Does cassava have much sugar in it when testing?,"Yes, but not as much as sugar cane"
22613,Can the chemical used for storing affect seed ...,"No, it does not affect germination"


In [7]:
df2.to_csv("second_data.csv",index=False)

In [19]:
df2.columns

Index(['question', 'answers'], dtype='object')

In [22]:
x = df.head(1)
print(x)

                                                text
0  <start_of_turn>What is the disease called when...


In [1]:
import pandas as pd
df2 = pd.read_csv("second_data.csv")

In [2]:
def template(question,answer):
    formatted = f"""You are a Agricultural Bot which will answer the following question ###Question : <start_of_turn>{question}<end_of_turn> ###Answer : <start_of_turn>{answer}<end_of_turn>"""
    return formatted

In [3]:
df2["text"] = df2.apply(lambda x: template(x["question"], x["answers"]), axis=1)
df2.columns

Index(['question', 'answers', 'text'], dtype='object')

In [4]:
df2 = df2.drop(['question', 'answers'], axis=1)
df2.columns
df2.head(1)

,text
0,You are a Agricultural Bot which will answer t...


In [5]:
x = df2.head(1)

In [6]:
print(x)

                                                text
0  You are a Agricultural Bot which will answer t...


In [7]:
df2.to_csv("second_data.csv",index=False)

In [ ]:
x = "<start_of_turn>What is the disease called when the leaf spot of banana is caused by Mycosphaerella spp?<end_of_turn><start_of_turn>The disease is called sigatoka.<end_of_turn>"

In [1]:
import pandas as pd


df = pd.read_csv("first_data.csv")


def extract_qa(text):
    question = ""
    answer = ""
    parts = text.split('<start_of_turn>')
    for part in parts:
        if '<end_of_turn>' in part:
            qa = part.split('<end_of_turn>')
            if not question:
                question = qa[0].strip()
            else:
                answer = qa[0].strip()
    return question, answer


df[['question', 'answer']] = df['text'].apply(lambda x: pd.Series(extract_qa(x)))


df.drop('text', axis=1, inplace=True)


print(df)

                                                question  \
0      What is the disease called when the leaf spot ...   
1                  How many types of sigatoka are there?   
2      Which species of sigatoka is more virulent and...   
3      Where is the leaf spot of banana widely distri...   
4      Which variety of banana is highly susceptible ...   
...                                                  ...   
21330             What is the purpose of seed treatment?   
21331                Is TV harmful to plants or animals?   
21332             What is the process of seed treatment?   
21333                    How is seedling treatment done?   
21334             What is the nursery treatment process?   

                                                  answer  
0                        The disease is called sigatoka.  
1      There are two types of sigatoka: yellow and bl...  
2      S.rolfsii is more virulent and progressively r...  
3      The leaf spot of banana is widely di

In [3]:
df.to_csv("new_first.csv",index=False)

In [4]:
def template(question,answer):
    formatted = f"""You are a Agricultural Bot which will answer the following question ###Question : <start_of_turn>{question}<end_of_turn> ###Answer : <start_of_turn>{answer}<end_of_turn>"""
    return formatted

In [6]:
df["text"] = df.apply(lambda x: template(x["question"], x["answer"]), axis=1)
df.columns

Index(['question', 'answer', 'text'], dtype='object')

In [7]:
df = df.drop(['question', 'answer'], axis=1)
df.columns
df.head(1)

,text
0,You are a Agricultural Bot which will answer t...


In [8]:
df.to_csv("first_data.csv",index=False)

In [2]:
import pandas as pd
df1 = pd.read_csv("first_data.csv")
df2 = pd.read_csv("second_data.csv")

print(df1.columns)
print(df2.columns)

Index(['text'], dtype='object')
Index(['text'], dtype='object')


In [3]:
df_combined = pd.concat([df1, df2], ignore_index=True)
print(len(df_combined))

43950


In [4]:
df_combined.to_csv("train.csv",index=False)

# **Training**

In [3]:
import pandas as pd
from datasets import Dataset

df = pd.read_csv("train.csv")
print(df)
formatted_data = Dataset.from_pandas(df)
print(formatted_data)

                                                    text
0      You are a Agricultural Bot which will answer t...
1      You are a Agricultural Bot which will answer t...
2      You are a Agricultural Bot which will answer t...
3      You are a Agricultural Bot which will answer t...
4      You are a Agricultural Bot which will answer t...
...                                                  ...
43945  You are a Agricultural Bot which will answer t...
43946  You are a Agricultural Bot which will answer t...
43947  You are a Agricultural Bot which will answer t...
43948  You are a Agricultural Bot which will answer t...
43949  You are a Agricultural Bot which will answer t...

[43950 rows x 1 columns]
Dataset({
    features: ['text'],
    num_rows: 43950
})


In [4]:
#Importing Libs
from transformers import AutoModelForCausalLM , AutoTokenizer , TrainingArguments , BitsAndBytesConfig
from peft import get_peft_model , LoraConfig , prepare_model_for_kbit_training
from accelerate import Accelerator
from trl import SFTTrainer
import torch
from datasets import load_dataset , Dataset
import pandas as pd

In [5]:
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0" #Tiny Llama - takes max of 3GB when training

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

#bnb-config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto" #"cpu" -change if necessary
)
model.config.use_cache = False #we are not inferencing so cache set to False
model.config.pretraining_tp = 1
model = prepare_model_for_kbit_training(model)

In [6]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear4bit(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMS

In [7]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout = 0.05,
    bias="none",
    task_type="CAUSAL_LM",
    #trains 0.1023 percent only without mentioning target_modules=["q_proj"]
)
model = get_peft_model(model,lora_config)
model.print_trainable_parameters()

trainable params: 1,126,400 || all params: 1,101,174,784 || trainable%: 0.1023


In [8]:
training_args = TrainingArguments(
    output_dir = "Tiny-Llama-AgriBot-FineTuned",
    overwrite_output_dir=True,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=3,
    optim="paged_adamw_32bit",
    learning_rate = 2e-4,
    lr_scheduler_type="cosine",
    save_strategy="steps",
    save_steps=1000,
    logging_steps=10,
    num_train_epochs=3, #increase the epochs
    fp16=True,
    push_to_hub=True
)

In [9]:
trainer = SFTTrainer(
    model=model,
    train_dataset = formatted_data,
    dataset_text_field="text",
    peft_config=lora_config,
    args=training_args,
    tokenizer = tokenizer,
    packing=False,
    max_seq_length=1024
)
trainer.train()

Map:   0%|          | 0/43950 [00:00<?, ? examples/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: siddharthmagesh007 (velammal-edu-in). Use `wandb login --relogin` to force relogin


  0%|          | 0/10988 [00:00<?, ?it/s]

d:\newenv\lib\site-packages\torch\utils\checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
d:\newenv\lib\site-packages\transformers\models\llama\modeling_llama.py:649: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


{'loss': 2.7136, 'grad_norm': 1.776272177696228, 'learning_rate': 0.00019999959127410606, 'epoch': 0.0}
{'loss': 2.066, 'grad_norm': 1.995184302330017, 'learning_rate': 0.00019999836509976534, 'epoch': 0.0}
{'loss': 1.6006, 'grad_norm': 2.3673996925354004, 'learning_rate': 0.00019999632148700122, 'epoch': 0.0}
{'loss': 1.1287, 'grad_norm': 1.789244294166565, 'learning_rate': 0.00019999346045251925, 'epoch': 0.0}


KeyboardInterrupt: 

# **Inference**

In [2]:
def template(question):
    formatted = f"""You are a Agricultural Bot which will answer the following question ###Question : <start_of_turn>{question}<end_of_turn> ###Answer : <start_of_turn> """
    return formatted

In [3]:
from transformers import AutoModelForCausalLM , AutoTokenizer

model_id = "pranesh16/Tiny-Llama-AgriBot"
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="cuda"
)
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
tokenizer.pad_token = tokenizer.eos_token

question = "what is crop rotation ? "
prompt = template(question)

inputs = tokenizer(prompt,return_tensors="pt").to('cuda')

output = model.generate(**inputs,max_new_tokens=256)

print(tokenizer.decode(output[0],skip_special_tokens=True))

config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

d:\newenv\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Siddharth\.cache\huggingface\hub\models--pranesh16--Tiny-Llama-AgriBot. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/2.41G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

d:\newenv\lib\site-packages\transformers\models\llama\modeling_llama.py:649: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


You are a Agricultural Bot which will answer the following question ###Question : <start_of_turn>what is crop rotation ? <end_of_turn> ###Answer : <start_of_turn> 3 or more crop seasons in a row<end_of_turn> ###Answer : <start_of_turn>
<end_of_turn> <end_of_turn> <start_of_turn> <end_of_turn> <end_of_turn> <end_of_turn> <end_of_turn> <end_of_turn> <end_of_turn> <end_of_turn> <end_of_turn> <end_of_turn> <end_of_turn> <end_of_turn> <end_of_turn> <end_of_turn> <end_of_turn> <end_of_turn> <end_of_turn> <end_of_turn> <end_of_turn> <end_of_turn> <end_of_turn> <end_of_turn> <end_of_turn> <end_of_turn> <end_of_turn> <end_of_turn> <end_of_turn> <end_of_turn> <end_of_turn> <end_of_turn> <end_of_
